In [8]:
from langchain_community.retrievers import BM25Retriever
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain.retrievers import EnsembleRetriever
import keyring

doc_list = [
    "우리나라는 2022년 코로나가 유행했다.",
    "우리나라 2024년 GDP 전망은 3.0%이다.",
    "우리나라는 2022년 국내총생산 중 연구개발 예산은 약 5%이다."
]


In [9]:

bm25_retriever = BM25Retriever.from_texts(
    doc_list, metadatas=[{"source":1}]*len(doc_list)
)
bm25_retriever.k = 1

embedding = OpenAIEmbeddings(api_key=keyring.get_password('openai', 'key_for_windows'))
chroma_vectorstore = Chroma.from_texts(
    doc_list, embedding, metadatas=[{'source':1}] * len(doc_list)
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwarg={'k':1})
faiss_vectorstore = FAISS.from_texts(
    doc_list, embedding, metadatas=[{'source':1}] * len(doc_list)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={'k':1})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever, faiss_retriever],
    weights=[0.2, 0.4, 0.4]
)

In [10]:
query = "2022년 우리나라 GDP대비 R&D 규모는?"

ensemble_docs = ensemble_retriever.invoke(query)
print(ensemble_docs)

[Document(metadata={'source': 1}, page_content='우리나라는 2022년 국내총생산 중 연구개발 예산은 약 5%이다.'), Document(metadata={'source': 1}, page_content='우리나라 2024년 GDP 전망은 3.0%이다.')]
